# Trabalho Prático 2 - Estruturas Criptográficas
## Mestrado em Engenharia Informática
### 28 de Março de 2023

Grupo 13, constituído por:
+ Rodrigo Pires Rodrigues, PG50726
+ Rui Guilherme Monteiro, PG50739

# Exercício 1
1. Construir uma classe Python que implemente um KEM - ElGamal. A classe deve:
    1. Inicializar cada instância recebendo  o parâmetro de segurança (tamanho em bits da ordem do grupo cíclico) e gere as chaves pública e privada.
    2. Conter funções para encapsulamento e revelação da chave gerada.
    3. Construir,  a partir deste KEM e usando a transformação de Fujisaki-Okamoto, um PKE que seja IND-CCA seguro.


## Imports

São importados os vários módulos necessários para o funcionamento do programa.

In [1]:
import os
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from sage.all import *
import hashlib
import hmac

# Estrutura geral
O código é dividido em três classes: KEM_ElGamal, PKE_FO (criptografia de chave pública usando a transformação de Fujisaki-Okamoto) e uma função principal main().



## Função get_random_prime

Utilizada para receber um número primo aleatório que será usada pela class "KEM_ElGamal".

In [2]:
def get_random_prime(lower_bound, upper_bound):
    return random_prime(upper_bound - 1, lbound=lower_bound)

## Classe KEM_ElGamal (key encapsulation mechanism)

A classe KEM_ElGamal implementa o algoritmo de criptografia de chave pública ElGamal para encapsular uma chave assimétrica, baseado na troca de chaves Diffie-Hellman. A chave privada é gerada aleatoriamente e a chave pública é composta por três parâmetros: um primo p, um gerador g e y = g^x (mod p), onde x é a chave privada. O método encaps() gera uma chave de sessão aleatória, realiza o cálculo de r e s = h^(xr) (mod p), onde h é o valor y recebido como chave pública, e retorna um par (s, c1), onde c1 = g^r (mod p). O método decaps() recebe como entrada um par (s, c1) e calcula a chave de sessão a partir da chave privada e de c1.

In [3]:
class KEM_ElGamal:
    def __init__(self, param):
        self.p = get_random_prime(2**(param-1), 2**param)
        self.g = primitive_root(self.p)
        self.x = randint(1, self.p-1)
        self.y = pow(self.g, self.x, self.p)
        self.param = param
        # Public key: (p, g, y)
        self.public_key = (self.p, self.g, self.y)
        # Private key: x
        self.private_key = self.x

    def encaps(self, pub_key):
        p, g, h = pub_key
        # r < p-1
        r = ZZ.random_element(p-1)
        s = pow(h, self.private_key * r, p)
        c1 = pow(g, r, p)
        return (s, c1)

    def decaps(self, ct):
        s, c1 = ct
        a = self.private_key
        k = pow(c1, a, self.p)
        k = k % self.p
        return k

## Classe PKE_FO (Public Key Encryption)
A classe PKE_FO implementa o esquema de criptografia de chave pública usando a transformação Fujisaki-Okamoto. O construtor recebe uma instância do KEM_ElGamal. O método encrypt() cifra uma mensagem usando o KEM_ElGamal e a transformação FO. 

Primeiramente, a chave de sessão é encapsulada usando o método encaps() do KEM. Em seguida, a chave é usada para derivar a chave AES-GCM e a chave de MAC. A mensagem é cifrada usando AES-GCM e o MAC é gerado usando a chave de MAC. O resultado final consiste numa tupla (kem_ct, ciphertext, mac, nonce), onde kem_ct é o resultado do encapsulamento da chave, ciphertext é o texto cifrado, mac é o valor do MAC e nonce é um valor aleatório usado para AES-GCM. O método decrypt() decifra uma mensagem usando a chave privada e a transformação Fujisaki-Okamoto. O método usa a chave privada do KEM para decapsular a chave de sessão, a qual é usada para derivar as chaves AES-GCM e de MAC. A mensagem é então descriptografada e o MAC é verificado. Se o MAC for válido, o método retorna o texto plano, caso contrário, uma exceção é gerada.

In [4]:
class PKE_FO:
    def __init__(self, kem):
        self.kem = kem

    def _hash(self, *args):
        # Hash function used in the Fujisaki-Okamoto transform
        h = hashlib.sha256()
        for arg in args:
            h.update(str(arg).encode('utf-8'))
        return h.digest()

    def encrypt(self, pub_key, msg):
        # Encrypt a message using the Fujisaki-Okamoto transform
        kem_pub_key = self.kem.public_key
        # Encapsulate the symmetric key using the ElGamal ephemeral key pair
        kem_ct = self.kem.encaps(pub_key)
        shared_key = self.kem.decaps(kem_ct)
        # Derive AES-GCM key and MAC key from the shared symmetric key
        aes_key = self._hash(shared_key, 0)
        mac_key = self._hash(shared_key, 1)
        # Encrypt the message using AES-GCM
        aes_gcm = AESGCM(aes_key)
        nonce = os.urandom(12)
        ciphertext = aes_gcm.encrypt(nonce, msg, None)
        # Compute the MAC of the ciphertext
        mac = hmac.new(mac_key, ciphertext, hashlib.sha256).digest()
        # Combine the ElGamal ciphertext and the AES-GCM ciphertext and MAC
        return (kem_ct, ciphertext, mac, nonce)


    def decrypt(self, priv_key, ct):
        # Decrypt a message using the Fujisaki-Okamoto transform
        kem_ct, ciphertext, mac, nonce = ct
        shared_key = self.kem.decaps(kem_ct)
        aes_key = self._hash(shared_key, 0)
        mac_key = self._hash(shared_key, 1)
        aes_gcm = AESGCM(aes_key)
        plaintext = aes_gcm.decrypt(nonce, ciphertext, None)
        if hmac.compare_digest(mac, hmac.new(mac_key, ciphertext, hashlib.sha256).digest()):
            return plaintext
        else:
            raise Exception("MAC verification failed")


## Main()

A função principal main() cria duas instâncias do KEM_ElGamal para Alice e Bob e usa a instância de Alice para criar uma instância de PKE_FO. A chave pública de Alice é compartilhada com Bob. Bob criptografa uma mensagem usando a chave pública de Alice e envia o texto criptografado para Alice. Alice usa sua chave privada para descriptografar a mensagem e imprime o texto plano na tela.

In [5]:
def main():
    # Generate Alice and Bob's public and private keys
    Bob_elgamal = KEM_ElGamal(128)
    Alice_elgamal = KEM_ElGamal(128)

    # Create PKE instance using Alice's KEM
    alice_pke_fo = PKE_FO(Alice_elgamal)

    # Alice shares her public key with Bob
    alice_pub_key = Alice_elgamal.public_key

    # Bob encrypts a message to Alice using Alice's public key
    message = b"Enemy attacks tonight!"
    ciphertext = alice_pke_fo.encrypt(alice_pub_key, message)

    # Alice decrypts the message
    plaintext = alice_pke_fo.decrypt(Alice_elgamal.private_key, ciphertext)

    # Print the decrypted message
    print("Decrypted message:", plaintext.decode())    

main()

Decrypted message: Enemy attacks tonight!
